In [1]:
import numpy as np

$C_r$：水の比熱(specific heat of water)[J/(kg・K)]

In [2]:
def Cr():
    return 4.18605E+3

$\rho_w$：水の密度(density of water)[kg/m3]

In [3]:
def roww():
    return 1000.0

## 熱・水分の収支計算をするモジュール  

あるセルの情報とセルに流出入する流量、時間差分を入力することで微小時間後の熱・水分の収支を計算するモジュール。  

流量に関してはmns側はセルに流入する方向が正、pls側はセルから流出する方が正、となっている。

熱に関しては局所平衡が成り立っていることが前提となり、固相と液相・気相の温度は一致する。  
また、蒸発潜熱については水分の蒸発量を水蒸気の収支量から算出することで計算した。

水分の収支計算は、

- 含水率ベース（高含水率域）
- 水分化学ポテンシャルベース（中間）
- 水蒸気圧ベース（低含水率域）

の3種類が一般的であるが、含水状態によってそれぞれ使い分けが必要である。  
それぞれの特徴については私（高取）の知る限りでは、以下のような認識である。

- 含水率ベース：低含水域においては変化量が非常に小さいため使用が困難。
- 水分化学ポテンシャルベース：広い範囲で使用可能だが、非常に高含水な領域（RH>0.99999、miu>-7）や非常に低含水な領域（RH<0.2）のような状況下では$\frac{\partial\mu}{\partial \psi}$が発散するため計算が困難になる。
- 水蒸気圧ベース：高含水域においては発散しやすくなる。

そのため、ここでは水分化学ポテンシャルベースでの収支計算を基本としクラスを作成した。


In [12]:
class MiuBased():

    def __init__( self, cell, flux_mns, flux_pls, dt ):
        self.cell     = cell
        self.dt       = dt
        self.flux_mns = flux_mns
        self.flux_pls = flux_pls
         
    # 熱収支計算
    def cal_ntemp( self ):
        if hasattr(self.cell, 'water'):
            crow_self = self.cell.material.crow + Cr() * roww() * self.cell.water.phi
            W_self    = -(self.flux_mns.jv - self.flux_pls.jv)
        else:
            crow_self = self.cell.material.crow
            W_self    = 0.0
        return cal_newtemp(crow = crow_self,\
                           temp = self.cell.material.temp,\
                           dq   = self.flux_mns.qs - self.flux_pls.qs,\
                           W    = W_self,\
                           dx   = self.cell.dx,\
                           dt   = self.dt )
    
    temp = property(cal_ntemp)
        
    # 水分収支計算
    def cal_nmiu( self ):
        return cal_newmiu(dphi = self.cell.water.dphi,\
                          miu  = self.cell.water.miu,\
                          djw  = ( self.flux_mns.jv + self.flux_mns.jl ) - ( self.flux_pls.jv + self.flux_pls.jl ),\
                          dx   = self.cell.dx,\
                          dt   = self.dt )
    
    miu = property(cal_nmiu)
    

### 熱収支式  

  
基礎方程式：$c\rho\frac{\partial T}{\partial t} = -\nabla・q + rW$  

時間差分解：$T_{t+1} = T_{t} + \frac{(q(x)-q(x+1) - rW )}{dx}\frac{dt}{c\rho}$  


In [5]:
def ratent_heat( temp ):
    return ( 597.5 - 0.559 * ( temp - 273.16 ) ) * 4186.05

In [10]:
#使用例
ratent_heat( 293.15 )

2454388.2360195

In [6]:
def cal_newtemp( crow, temp, dq, W, dx, dt ):
    return temp + ( dq - ratent_heat( temp ) * W ) / dx * ( dt / crow )

In [11]:
# 使用例
cal_newtemp( crow = 1109437.5, temp = 283.15, dq = -45.0, W = -1.7861334003233654e-07, dx = 0.1, dt = 0.01 )

283.1499959837815


### 水分収支式  
基礎方程式：$\frac{\partial[(\phi_{0}-\psi)\rho_{v}+\rho_{w}\psi]}{\partial t} = -\nabla・J_w$  

$\phi_{0}$：絶乾時の材料の空隙率[-]  
$\psi$：含水率[-]  
$\phi_{0}-\psi$：含水率$\psi$の時の空隙率[-]  
$\rho_w, \rho_v$：それぞれ液水、水蒸気の密度[kg/m3]  
$J_w$：液水の流量

#### 水分化学ポテンシャルベース  

$\rho_{w}\frac{\partial\psi}{\partial \mu}\frac{\partial\mu}{\partial t} = -\nabla・J_w$  

時間差分解：$\mu_{t+1} = \mu_{t} + \frac{J_w(x) -J_w(x+1)}{dx} \frac{\partial\mu}{\partial \psi} \frac{dt}{\rho_{w}}$  

In [7]:
def cal_newmiu( dphi, miu, djw, dx, dt ):
    return miu + djw / dx / dphi * ( dt / roww() )


#### 含水率ベース（高含水域）  
$\rho_w >> \rho_v,　\rho_w \approx const$、より高含水域では、水分の収支式を以下のように近似できる。  

$\rho_{w}\frac{\partial\psi}{\partial t} = -\nabla・J_w$  

時間差分解：$\phi_{t+1} = \phi_{t} - \frac{J_w(x) -J_w(x+1)}{dx} \frac{dt}{\rho_{w}}$

In [8]:
def cal_newphi( phi, djw, dx, dt ):
    return phi + djw / dx * ( dt / roww() )


#### ハイグロスコピック（蒸気拡散支配領域）    

$(\phi_0\gamma'+\kappa) \frac{\partial X}{\partial t} - \nu \frac{\partial T}{\partial t} = \lambda'_x\frac{\partial^2 X}{\partial x^2}$  

$- r\kappa \frac{\partial X}{\partial t}  + (c\rho +r\nu) \frac{\partial T}{\partial t} = \lambda \frac{\partial^2 T}{\partial x^2}$  

ただし  
$\kappa = \rho_w (\frac{\partial \psi}{\partial X})_T$,　  $\nu = \rho_w(\frac{\partial \psi}{\partial T})_X$  

$\gamma'$：乾燥空気の密度[kg/m3]